In [ ]:
import pandas as pd
import numpy as np
import tensorflow as tf
from sklearn.preprocessing import MinMaxScaler
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, Dense

# Load dataset
df = pd.read_csv("IMU.csv")  # Replace with your file path

# Select features (IMU data)
features = ["ax", "ay", "az", "wx", "wy", "wz", "Bx", "By", "Bz"]
target = ["latitude", "longitude"]

# Normalize data
scaler_x = MinMaxScaler()
scaler_y = MinMaxScaler()
df[features] = scaler_x.fit_transform(df[features])
df[target] = scaler_y.fit_transform(df[target])

# Convert to sequences for LSTM
sequence_length = 10  # Use last 10 readings to predict next GPS
X, Y = [], []
for i in range(len(df) - sequence_length):
    X.append(df[features].iloc[i:i+sequence_length].values)
    Y.append(df[target].iloc[i+sequence_length].values)

X, Y = np.array(X), np.array(Y)

# Split data (80% train, 20% test)
train_size = int(0.8 * len(X))
X_train, X_test = X[:train_size], X[train_size:]
Y_train, Y_test = Y[:train_size], Y[train_size:]


In [3]:
# Define LSTM Model
model = Sequential([
    LSTM(50, return_sequences=True, input_shape=(sequence_length, len(features))),
    LSTM(50),
    Dense(20, activation='relu'),
    Dense(2)  # Output: Latitude & Longitude
])

model.compile(optimizer='adam', loss='mse')
model.fit(X_train, Y_train, epochs=50, batch_size=16, validation_data=(X_test, Y_test))

# Save the trained model
model.save("gps_predictor.h5")


Epoch 1/50


c:\Users\G15\Desktop\coding\.conda\Lib\site-packages\keras\src\layers\rnn\rnn.py:204: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(**kwargs)


515/515 ━━━━━━━━━━━━━━━━━━━━ 4s 4ms/step - loss: 0.0811 - val_loss: 0.0551
Epoch 2/50
515/515 ━━━━━━━━━━━━━━━━━━━━ 1s 3ms/step - loss: 0.0585 - val_loss: 0.0592
Epoch 3/50
515/515 ━━━━━━━━━━━━━━━━━━━━ 2s 3ms/step - loss: 0.0585 - val_loss: 0.0532
Epoch 4/50
515/515 ━━━━━━━━━━━━━━━━━━━━ 1s 3ms/step - loss: 0.0583 - val_loss: 0.0625
Epoch 5/50
515/515 ━━━━━━━━━━━━━━━━━━━━ 2s 3ms/step - loss: 0.0584 - val_loss: 0.0676
Epoch 6/50
515/515 ━━━━━━━━━━━━━━━━━━━━ 1s 3ms/step - loss: 0.0575 - val_loss: 0.0680
Epoch 7/50
515/515 ━━━━━━━━━━━━━━━━━━━━ 2s 3ms/step - loss: 0.0564 - val_loss: 0.0697
Epoch 8/50
515/515 ━━━━━━━━━━━━━━━━━━━━ 1s 3ms/step - loss: 0.0545 - val_loss: 0.0719
Epoch 9/50
515/515 ━━━━━━━━━━━━━━━━━━━━ 1s 3ms/step - loss: 0.0530 - val_loss: 0.0828
Epoch 10/50
515/515 ━━━━━━━━━━━━━━━━━━━━ 1s 3ms/step - loss: 0.0519 - val_loss: 0.0904
Epoch 11/50
515/515 ━━━━━━━━━━━━━━━━━━━━ 2s 3ms/step - loss: 0.0505 - val_loss: 0.0910
Epoch 12/50
515/515 ━━━━━━━━━━━━━━━━━━━━ 1s 3ms/step - loss: 0.

In [ ]:
from tensorflow.keras.losses import MeanSquaredError

# Load model with custom loss function
model = tf.keras.models.load_model(
    "C:\\Users\\G15\\Desktop\\coding\\PYTHON\\INS\\gps_predictor.h5",
    custom_objects={'mse': MeanSquaredError()}
)

# Example: Predicting GPS when only IMU data is available
def predict_next_gps(last_imu_readings):
    last_imu_readings = scaler_x.transform(last_imu_readings)  # Normalize
    last_imu_readings = np.expand_dims(last_imu_readings, axis=0)  # Reshape for LSTM
    predicted_gps = model.predict(last_imu_readings)
    return scaler_y.inverse_transform(predicted_gps)[0]  # Convert back to original scale

# Example input (last 10000 IMU readings from the dataset)
last_imu_readings = df[features].iloc[-10000:].values
predicted_lat_lon = predict_next_gps(last_imu_readings)
print("Predicted GPS:", predicted_lat_lon)


c:\Users\G15\Desktop\coding\.conda\Lib\site-packages\sklearn\base.py:493: UserWarning: X does not have valid feature names, but MinMaxScaler was fitted with feature names
  warnings.warn(


1/1 ━━━━━━━━━━━━━━━━━━━━ 1s 554ms/step
Predicted GPS: [12.861916 77.66468 ]
